In [45]:
# Run this cell first to install packages
#!pip install langchain langchain-openai python-dotenv

In [39]:
# Test if your safe setup works
import os
from dotenv import load_dotenv

load_dotenv()

# Check if key loads (don't print the actual key!)
if os.getenv("OPENAI_API_KEY"):
    print("✅ API key loaded safely!")
else:
    print("❌ API key not found - check your .env file")

# Test with LangChain
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-3.5-turbo")
response = llm.invoke([HumanMessage(content="Say 'Security test passed'")])
print("Response:", response.content)

✅ API key loaded safely!
Response: 'Security test passed'


In [41]:
# Step 1: Create a simple function to ask questions
def ask_ai(question):
    message = HumanMessage(content=question)
    response = llm.invoke([message])
    return response.content

# Test it
answer = ask_ai("What is 2+2?")
print(answer)

2 + 2 = 4


In [43]:
# Step 2: Import template tools
from langchain.prompts import PromptTemplate

# Create a template
template = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple words:"
)

# Test the template
prompt = template.format(topic="Python")
print("Prompt:", prompt)

Prompt: Explain Python in simple words:


In [15]:
# Step 3: Use template with AI
def explain_topic(topic):
    # Make the prompt
    prompt = template.format(topic=topic)
    
    # Ask AI
    answer = ask_ai(prompt)
    return answer

# Test it
result = explain_topic("cats")
print("Topic: cats")
print("Answer:", result)

Topic: cats
Answer: Cats are furry animals that are commonly kept as pets. They are known for their agility, independence, and playful nature. Cats are carnivores, which means they primarily eat meat. They have retractable claws and sharp teeth, which they use for hunting and grooming. Cats are known for their ability to purr and meow to communicate with their owners. They are also known for their curiosity and love to explore their surroundings. Cats are generally clean animals and spend a lot of time grooming themselves. They can be very affectionate and form strong bonds with their human companions.


In [19]:
# Step 5: Modern way (no warnings)
# Chain = template + AI
modern_chain = template | llm

# Use it
result = modern_chain.invoke({"topic": "fish"})
print("Modern chain about fish:")
print(result.content)

Modern chain about fish:
Fish are cold-blooded animals that live in water. They have fins to help them swim and gills to breathe. Fish come in many different shapes, sizes, and colors, and they eat a variety of foods like plants, insects, and other fish. People enjoy fishing for fish as a sport or to eat.


In [23]:
# Step 7: Create nutrition template
nutrition_template = PromptTemplate(
    input_variables=["food"],
    template="List the nutritional value of {food}. Include calories, protein, carbs, and vitamins:"
)

# Create nutrition chain
nutrition_chain = nutrition_template | llm

# Test with pizza
result = nutrition_chain.invoke({"food": "pizza"})
print("Pizza nutrition:")
print(result.content)

Pizza nutrition:
The nutritional value of a typical slice of cheese pizza (1/8th of a 14-inch pizza) can vary depending on the type of crust and toppings used. On average, a slice of cheese pizza contains around:

- Calories: 200-300
- Protein: 8-12 grams
- Carbohydrates: 25-35 grams
- Fat: 8-12 grams
- Vitamins and minerals: Pizza can provide some essential vitamins and minerals, such as calcium, iron, vitamin A, and vitamin C, depending on the ingredients used.

It's important to note that the nutritional value can vary significantly based on the type of crust, cheese, and toppings used. Adding vegetables or lean proteins as toppings can increase the nutritional value of pizza.


In [25]:
# Step 1: Template with multiple variables
multi_template = PromptTemplate(
    input_variables=["style", "topic", "audience"],
    template="""
    Write about {topic} in a {style} style for {audience}.
    Keep it short and clear.
    """
)

# Create chain
style_chain = multi_template | llm

# Test it
result = style_chain.invoke({
    "style": "funny", 
    "topic": "cats", 
    "audience": "kids"
})

print("Multi-variable result:")
print(result.content)

Multi-variable result:
Cats are like little ninjas in fur coats. They can sneak up on you without making a sound, and then pounce on your shoelaces like they're hunting a wild beast. But don't let their cute faces fool you - they're really just plotting how to take over the world one nap at a time. So next time you see a cat, be sure to give them some treats and maybe they'll let you live to see another day.


In [27]:
# Step 2: Try different combinations
# Professional style
result1 = style_chain.invoke({
    "style": "professional", 
    "topic": "programming", 
    "audience": "beginners"
})

print("Professional style:")
print(result1.content)
print("\n" + "="*50 + "\n")

# Casual style
result2 = style_chain.invoke({
    "style": "casual", 
    "topic": "cooking", 
    "audience": "teenagers"
})

print("Casual style:")
print(result2.content)

Professional style:
Programming is the process of creating instructions for computers to follow. It involves writing code using programming languages such as Python, JavaScript, or Java. By learning how to program, you can develop software, websites, and applications.

To start programming, you need to understand basic programming concepts such as variables, loops, and functions. Practice coding regularly and seek feedback from experienced programmers to improve your skills.

Remember to write clean and organized code, comment on your code to explain your logic, and test your code thoroughly to ensure it works correctly. With dedication and practice, you can become a proficient programmer and build amazing projects.


Casual style:
Hey guys, let's talk about cooking! It may seem daunting at first, but trust me, it can be super fun and rewarding. Start with simple recipes like pasta or grilled cheese sandwiches. Don't be afraid to get creative and add your own twist to dishes. Remember 

In [29]:
# Step 3: Few-shot learning (teach by examples)
few_shot_template = PromptTemplate(
    input_variables=["word"],
    template="""
    Create a simple definition like these examples:
    
    Example 1:
    Word: Happy
    Definition: Feeling good and smiling
    
    Example 2:
    Word: Rain
    Definition: Water falling from clouds
    
    Now do this one:
    Word: {word}
    Definition:
    """
)

# Create chain
definition_chain = few_shot_template | llm

# Test it
result = definition_chain.invoke({"word": "computer"})
print("Few-shot definition:")
print(result.content)

Few-shot definition:
Machine that processes information and performs tasks according to instructions.


In [31]:
# Step 4: Conditional templates
conditional_template = PromptTemplate(
    input_variables=["age", "topic"],
    template="""
    Explain {topic} for someone who is {age} years old.
    
    If age is under 10: use very simple words and fun examples
    If age is 10-17: use clear explanations with relatable examples  
    If age is 18+: use detailed, technical explanations
    
    Age: {age}
    Topic: {topic}
    Explanation:
    """
)

# Create chain
age_chain = conditional_template | llm

# Test different ages
result1 = age_chain.invoke({"age": "5", "topic": "electricity"})
print("For 5-year-old:")
print(result1.content)
print("\n" + "="*30 + "\n")

result2 = age_chain.invoke({"age": "25", "topic": "electricity"})
print("For 25-year-old:")
print(result2.content)

For 5-year-old:
Okay kiddo, picture electricity like tiny invisible super fast cars running through wires. When we plug things in, the cars zoom through the wires to give our toys power to work and lights to shine bright. Just remember to always be careful around electricity and never touch wires or outlets. It's like magic, but we have to use it safely!


For 25-year-old:
Electricity is a form of energy that is created when charged particles, like electrons, flow through a conductor, such as a wire. These charged particles carry energy from power plants to our homes, allowing us to power our devices and appliances. Electricity is essential for modern life, as it powers everything from our lights and electronics to transportation systems and industrial machinery. It is generated through various methods, such as burning fossil fuels, harnessing the power of flowing water, or capturing energy from sunlight or wind. Understanding how electricity works helps us manage and utilize this vita